In [1]:
%reload_ext autotime
import pandas as pd
import plotly.io as pio
import plotly.express as px # Plotting
from tqdm import tqdm
tqdm.pandas()
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor
pio.renderers.default = 'notebook'
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 100)

time: 1.37 s (started: 2025-06-30 07:28:45 +12:00)


In [2]:
df = pd.read_excel("LLM_results.xlsx")
df = df[(df.timeLeft == "closed") & (df.goal > 0)]
df.sentiment = df.sentiment.astype(str).str.split(" ").str[0].astype(float)
df.truth = df.truth.astype(str).str.split(" ").str[0].astype(float)
df.urgency = df.urgency.astype(str).str.split(" ").str[0].astype(float)
df["Target_completion"] = df.amountRaised / df.goal * 100
df

,Index,title,pitch,hero,uri,amountRaised,goal,timeLeft,action,actionUri,start,end,n_questions,location,n_donors,description,use_of_funds,whos_involved,moderated,n_updates,updates,_21,text,condition,ICD10,ICD,name,gender,age,age_group,ethnicity,urgency,sentiment,truth,notes,smiling,deservingness,attractiveness,use,region,narrative_clarity,narrative_quality,emotional_tone,image_type,face_visible,facial_expression,image_quality,Target_completion
31,31,Helping a person in need,Helping out a person in need that would drop his hat to help someone else,https://static.givealittle.co.nz/assets/hero/ed760e80-7e10-4078-b7a8-b2ec0104419b-320:False,https://givealittle.co.nz/cause/helping-a-person-in-need,1700.0,2000.0,closed,Read more,https://givealittle.co.nz/cause/helping-a-person-in-need,29 May 2025,8 Jun 2025,0,"Blenheim, Marlborough",54.0,"Tok had a serious injury while running — he tore a muscle, which led to internal bleeding and ev...",medical care where needed and general stuff around the house if needed also and utility's,"Created by, and paying to a verified bank account of,\r\nMatthew bergman on behalf of Tok H...",Page Moderated,3.0,Big thanks\n7 June 2025\nHey team huge thanks to all that have donated Tok is humbled beyond wor...,NaN,Title: Helping a person in need\nPitch: Helping out a person in need that would drop his hat to ...,Muscle tear leading to internal bleeding and infection,S86.0,"Chapter S00-T88 - Injury, poisoning and certain other consequences of external causes",Tok,Male,Indeterminate,indeterminate/unknown,Māori (based on visual appearance),80.0,70.0,90.0,"Age was not explicitly mentioned in the text, so it is marked as indeterminate. Ethnicity was in...",True,90,60,"medical expenses, travel expenses",Marlborough,80,75,grateful,portrait,True,smiling,medium,85.000000
56,56,Please help Walter 🙏 to get the veterinary care he needs.,Please help us 🙏 Walty needs bloodwork to rule out cancer 😪,https://static.givealittle.co.nz/assets/hero/a5763829-1835-4780-adc3-b2e500bd5557-320:False,https://givealittle.co.nz/cause/please-help-walter-to-get-the-veterinary-care-he,380.0,500.0,closed,Read more,https://givealittle.co.nz/cause/please-help-walter-to-get-the-veterinary-care-he,22 May 2025,1 Jun 2025,0,Otago,14.0,"Walter has been my life for 15 years, since he was a pup. It has always being just the two of us...","The money raised will go into Walts vet costsFull bloods, medication etc at the moment is quoted...","Created by, and paying to a verified bank account of,\r\nKerri Back",Page Moderated,1.0,Bloodwork done.\n23 May 2025\nThank you so much for your kind donations.\nWalty has had his bloo...,NaN,Title: Please help Walter 🙏 to get the veterinary care he needs.\nPitch: Please help us 🙏 Walty ...,Pancreatitis,K85,Chapter K00-K95 - Diseases of the digestive system,Walter,Other/unknown,indeterminate/unknown,indeterminate/unknown,Unknown (animal),80.0,70.0,90.0,"The subject is a pet dog, not a human. Age and age group cannot be determined precisely.",True,90,80,medical expenses,Otago,85,80,grateful,portrait,True,smiling,high,76.000000
59,59,Medical Treatment for Atish Lal for Coronary Artery Disease,medical Treatment,https://static.givealittle.co.nz/assets/hero/0a81d0ab-2bfc-4890-8049-b2e3017fea04-320:False,https://givealittle.co.nz/cause/medical-treatment-for-atish-lal-for-coronary,380.0,14900.0,closed,Read more,https://givealittle.co.nz/cause/medical-treatment-for-atish-lal-for-coronary,20 May 2025,6 Jun 2025,0,Auckland,6.0,"Dear Family and Friends,\nI hope this message finds you well.\nI am reaching out today with a hu...","Estimated costs for Treatment & Travel:- PTCA treatment package + home stay : $11,000 USD- Reser...",Created by\r\nShaneel Prasad \n\n\n\n\r\n Paying to a verified bank account of\r\nPriya Go...,Page Moderated,NaN,NaN,NaN,Title: Medical Treatment for Atish Lal for Coronary Artery Disease\nPitch: medical Treatment\nDe...,Coronary Artery Disease,I25.1,Chapter I00-I99 - Diseases of the circulatory syst

time: 10.2 s (started: 2025-06-30 07:28:47 +12:00)


In [3]:
df.gender.value_counts().head(20)

gender
Female                                                                                2276
Male                                                                                  1648
Other/unknown                                                                          542
Non-binary                                                                               3
Male and Female                                                                          3
Mixed (Male and Female)                                                                  2
Female (target audience is primarily women, but includes non-binary and trans men)       1
Mixed (group of individuals)                                                             1
Female (Lesa), Male (Victor)                                                             1
Other/unknown (as they are twin babies)                                                  1
Other/unknown (animal)                                                             

time: 5.38 ms (started: 2025-06-30 07:28:57 +12:00)


In [4]:
print("\n".join(df[df.gender == "Male and Female"].uri.to_list()))

https://givealittle.co.nz/cause/brook-smith-family
https://givealittle.co.nz/cause/cancellation-of-second-street-appeal
https://givealittle.co.nz/cause/help-matt-and-jiwoo-get-out-of-their-rough-patch
time: 1.55 ms (started: 2025-06-30 07:28:57 +12:00)


In [5]:
df.gender = df.gender.str.lower().where(df.gender.str.lower().isin(['male', 'female']), 'other')
df.gender.value_counts()

gender
female    2276
male      1648
other      575
Name: count, dtype: int64

time: 9.2 ms (started: 2025-06-30 07:28:57 +12:00)


In [6]:
df.ethnicity.value_counts().head(20)

ethnicity
Caucasian (assumed based on image)                 1499
European/Caucasian (based on image)                 216
Caucasian (based on image)                          213
European/New Zealander (based on image)             162
Caucasian (assumed from image)                       69
Pacific Islander (based on image)                    69
South Asian (based on image)                         69
Filipino                                             68
South Asian                                          67
Caucasian                                            61
Asian (based on image)                               56
Māori                                                53
European/New Zealander (assumed based on image)      53
European/Caucasian (assumed based on image)          53
Indian                                               49
European/New Zealand European (based on image)       45
Māori (based on visual appearance)                   44
Asian                                 

time: 7.85 ms (started: 2025-06-30 07:28:57 +12:00)


In [7]:
df.ethnicity = df.ethnicity.str.extract(r"(.+?)(?= \(|$)")
df.ethnicity.value_counts().head(20)

ethnicity
Caucasian                        1886
European/New Zealander            298
Māori                             289
European/Caucasian                281
South Asian                       184
Asian                             178
Pacific Islander                  128
Unknown                            82
Filipino                           78
European/New Zealand               73
Indian                             69
European/New Zealand European      54
Maori                              47
African                            40
Not specified                      36
Not applicable                     34
Indeterminate/Unknown              33
Mixed                              32
Latin American/Hispanic            29
Māori/Pacific Islander             28
Name: count, dtype: int64

time: 26.2 ms (started: 2025-06-30 07:28:57 +12:00)


In [8]:
df.ethnicity = df.ethnicity.replace({
    "European/Caucasian": "Caucasian",
    "European/White": "Caucasian",
    "European/White": "Caucasian",
    "European/New Zealand": "Caucasian",
    "European/New Zealander": "Caucasian",
    "European/New Zealand European": "Caucasian",
    "Māori/Pacific Islander": "Māori",
    "Asian/Pacific Islander": "Pacific Islander",
    "Maori": "Māori"
})
top_20 = df.ethnicity.value_counts().head(20)
display(top_20)
df.ethnicity = df.ethnicity.where(df.ethnicity.isin(top_20.index), "other")

ethnicity
Caucasian                     2594
Māori                          364
South Asian                    184
Asian                          178
Pacific Islander               153
Unknown                         82
Filipino                        78
Indian                          69
African                         40
Not specified                   36
Not applicable                  34
Indeterminate/Unknown           33
Mixed                           32
Latin American/Hispanic         29
Fijian                          24
Indian/Fijian                   24
East Asian                      22
Fijian/Pacific Islander         20
European/New Zealand White      19
Indeterminate                   17
Name: count, dtype: int64

time: 15.7 ms (started: 2025-06-30 07:28:57 +12:00)


In [9]:
df.condition.str.lower().value_counts().head(20)

condition
cancer                     185
breast cancer              143
gender dysphoria           137
stroke                      75
multiple sclerosis          67
none specified              54
endometriosis               45
bowel cancer                39
type 1 diabetes             37
melanoma                    37
mental health issues        36
cerebral palsy              33
terminal cancer             33
infertility                 33
chronic kidney disease      31
obesity                     31
heart attack                29
coronary artery disease     28
lung cancer                 27
ovarian cancer              27
Name: count, dtype: int64

time: 7.52 ms (started: 2025-06-30 07:28:57 +12:00)


In [10]:
df.sentiment.value_counts()

sentiment
70.0     1860
60.0      713
50.0      657
80.0      453
85.0      268
30.0      215
90.0      182
75.0       84
40.0       29
20.0       23
95.0        8
10.0        6
100.0       1
Name: count, dtype: int64

time: 4.92 ms (started: 2025-06-30 07:28:57 +12:00)


In [11]:
df.use.str.lower().value_counts().head(20)

use
medical expenses                                                         2054
medical expenses, travel expenses                                         624
travel expenses                                                           327
medical expenses, travel expenses, lost wages                             306
medical expenses, experimental therapies                                  238
medical expenses, lost wages                                              221
medical expenses, experimental therapies, travel expenses                  93
medical expenses, experimental therapies, travel expenses, lost wages      77
travel expenses, lost wages                                                43
experimental therapies                                                     42
travel expenses, medical expenses                                          31
experimental therapies, travel expenses                                    27
medical expenses, travel expenses, accommodation            

time: 14.1 ms (started: 2025-06-30 07:28:57 +12:00)


In [12]:
df.age_group = df.age_group.str.extract(r"(.+?)(?= \(|$)")
df.age_group = df.age_group.str.lower().where(df.age_group.str.lower().isin(['0-14', '15-64', '65+']), 'indeterminate/unknown')
df.age_group.value_counts()

age_group
indeterminate/unknown    2671
15-64                    1117
0-14                      504
65+                       207
Name: count, dtype: int64

time: 38.3 ms (started: 2025-06-30 07:28:57 +12:00)


In [13]:
df.gender.value_counts()

gender
female    2276
male      1648
other      575
Name: count, dtype: int64

time: 8.28 ms (started: 2025-06-30 07:28:57 +12:00)


In [14]:
df["Charity"] = df.whos_involved.str.contains(r"(Charity)", regex=False)

time: 7.97 ms (started: 2025-06-30 07:28:57 +12:00)


In [15]:
df.to_excel("LLM_results_cleaned.xlsx", index=False)

time: 6.69 s (started: 2025-06-30 07:28:57 +12:00)


In [16]:
df["met_target"] = (df.amountRaised >= df.goal).astype(int)

time: 1.61 ms (started: 2025-06-30 07:29:04 +12:00)


In [17]:
def parse(age):
    try:
        return float(age)
    except ValueError:
        return None

df["parsed_age"] = df.age.apply(parse)
df.parsed_age.value_counts().head(20)

parsed_age
25.0    52
30.0    51
35.0    47
17.0    43
20.0    41
28.0    39
23.0    37
6.0     34
40.0    33
27.0    33
18.0    32
4.0     32
13.0    30
3.0     28
26.0    28
38.0    27
19.0    27
14.0    27
2.0     26
50.0    26
Name: count, dtype: int64

time: 9.05 ms (started: 2025-06-30 07:29:04 +12:00)


In [18]:
df.met_target.value_counts()

met_target
0    3859
1     640
Name: count, dtype: int64

time: 4.3 ms (started: 2025-06-30 07:29:04 +12:00)


In [19]:
df.deservingness = df.deservingness.apply(parse).astype(float)
df.deservingness.describe()

count    4499.000000
mean       91.133585
std         4.859491
min        20.000000
25%        90.000000
50%        90.000000
75%        95.000000
max       100.000000
Name: deservingness, dtype: float64

time: 9.05 ms (started: 2025-06-30 07:29:04 +12:00)


In [20]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, chi2_contingency, pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

def analyze_feature_influence(df, target_col='met_target'):
    results = {}
    target = df[target_col]
    is_target_numeric = np.issubdtype(target.dtype, np.number)

    for col in df.columns:
        if col == target_col:
            continue
        
        feature = df[col]
        if feature.isnull().any() or target.isnull().any():
            continue  # skip missing for simplicity

        if is_target_numeric:
            # Target is numeric
            if np.issubdtype(feature.dtype, np.number):
                # Pearson correlation
                corr, p_value = pearsonr(feature, target)
                results[col] = {
                    'test': 'pearsonr',
                    'stat': corr,
                    'p_value': p_value
                }
            else:
                # ANOVA (feature is categorical)
                groups = [target[feature == val] for val in feature.unique()]
                stat, p_value = f_oneway(*groups)
                results[col] = {
                    'test': 'anova',
                    'stat': stat,
                    'p_value': p_value
                }
        else:
            # Target is categorical
            if not np.issubdtype(feature.dtype, np.number):
                # Chi-squared test
                contingency = pd.crosstab(df[col], target)
                stat, p, _, _ = chi2_contingency(contingency)
                results[col] = {
                    'test': 'chi2',
                    'stat': stat,
                    'p_value': p
                }

    return pd.DataFrame(results).T.sort_values('p_value')

# Usage:
# df = pd.read_csv('your_data.csv')
df.met_target = df.met_target.astype(bool)
analyze_feature_influence(df, target_col='met_target')

,test,stat,p_value
image_quality,chi2,40.932891,0.0
emotional_tone,chi2,73.290279,0.0
age_group,chi2,21.177083,0.000097
facial_expression,chi2,26.442983,0.000184
ethnicity,chi2,48.378961,0.000376
end,chi2,2186.22469,0.215375
region,chi2,104.213501,0.290067
notes,chi2,2766.80732,0.292727
start,chi2,2427.549619,0.326723
whos_involved,chi2,4433.025955,0.417108


time: 681 ms (started: 2025-06-30 07:29:04 +12:00)


In [21]:
df.smiling = df.smiling.astype(bool)

time: 868 μs (started: 2025-06-30 07:29:05 +12:00)


In [22]:
np.exp(.66)

np.float64(1.9347923344020317)

time: 6.82 ms (started: 2025-06-30 07:29:05 +12:00)


In [23]:
df["have_age"] = ~df.parsed_age.isna()
df[["met_target", "have_age"]].value_counts()

met_target  have_age
False       False       2509
            True        1350
True        False        440
            True         200
Name: count, dtype: int64

time: 8.17 ms (started: 2025-06-30 07:29:05 +12:00)


In [24]:
formula = "met_target ~ sentiment + smiling + gender + truth + urgency + parsed_age"
logit = smf.glm(formula=formula, data=df, family=sm.families.Binomial(link=sm.families.links.Logit())).fit()
display(logit.summary())
display(f"AIC (Logit): {logit.aic:.2f}")
probit = smf.glm(formula=formula, data=df, family=sm.families.Binomial(link=sm.families.links.Probit())).fit()
display(probit.summary())
display(f"AIC (Probit): {probit.aic:.2f}")

<class 'statsmodels.iolib.summary.Summary'>
"""
                             Generalized Linear Model Regression Results                             
=====================================================================================================
Dep. Variable:     ['met_target[False]', 'met_target[True]']   No. Observations:                 1550
Model:                                                   GLM   Df Residuals:                     1542
Model Family:                                       Binomial   Df Model:                            7
Link Function:                                         Logit   Scale:                          1.0000
Method:                                                 IRLS   Log-Likelihood:                -575.14
Date:                                       Mon, 30 Jun 2025   Deviance:                       1150.3
Time:                                               07:29:05   Pearson chi2:                 1.55e+03
No. Iterations:                                            5   Pseudo R-squ. (CS):            0.02661
Covariance Type:                                   nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.5947      2.473      2.667      0.008       1.748      11.442
smiling[T.True]    -0.3659      0.171     -2.146      0.032      -0.700      -0.032
gender[T.male]     -0.2366      0.158     -1.494      0.135      -0.547       0.074
gender[T.other]     0.5258      0.447      1.177      0.239      -0.350       1.401
sentiment          -0.0143      0.007     -1.953      0.051      -0.029    5.17e-05
truth              -0.0395      0.030     -1.302      0.193      -0.099       0.020
urgency            -0.0033      0.011     -0.305      0.761      -0.024       0.018
parsed_age          0.0173      0.004      3.860      0.000       0.009       0.026
===================================================================================
"""

'AIC (Logit): 1166.28'

<class 'statsmodels.iolib.summary.Summary'>
"""
                             Generalized Linear Model Regression Results                             
=====================================================================================================
Dep. Variable:     ['met_target[False]', 'met_target[True]']   No. Observations:                 1550
Model:                                                   GLM   Df Residuals:                     1542
Model Family:                                       Binomial   Df Model:                            7
Link Function:                                        Probit   Scale:                          1.0000
Method:                                                 IRLS   Log-Likelihood:                -575.30
Date:                                       Mon, 30 Jun 2025   Deviance:                       1150.6
Time:                                               07:29:05   Pearson chi2:                 1.55e+03
No. Iterations:                                            6   Pseudo R-squ. (CS):            0.02641
Covariance Type:                                   nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.5673      1.288      2.770      0.006       1.043       6.091
smiling[T.True]    -0.1824      0.090     -2.034      0.042      -0.358      -0.007
gender[T.male]     -0.1288      0.085     -1.509      0.131      -0.296       0.039
gender[T.other]     0.2695      0.223      1.210      0.226      -0.167       0.706
sentiment          -0.0074      0.004     -1.920      0.055      -0.015       0.000
truth              -0.0214      0.016     -1.355      0.175      -0.052       0.010
urgency            -0.0010      0.006     -0.185      0.853      -0.012       0.010
parsed_age          0.0093      0.002      3.967      0.000       0.005       0.014
===================================================================================
"""

'AIC (Probit): 1166.59'

time: 98 ms (started: 2025-06-30 07:29:05 +12:00)


In [25]:
df.met_target = df.met_target.astype(int)
smf.ols(formula="Target_completion ~ sentiment + smiling + gender + truth + urgency + parsed_age", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Target_completion   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.6657
Date:                Mon, 30 Jun 2025   Prob (F-statistic):              0.701
Time:                        07:29:05   Log-Likelihood:                -13128.
No. Observations:                1550   AIC:                         2.627e+04
Df Residuals:                    1542   BIC:                         2.631e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        -943.6357    816.318     -1.156      0.248   -2544.846     657.575
smiling[T.True]    50.2080     62.775      0.800      0.424     -72.925     173.341
gender[T.male]     97.3301     61.465      1.584      0.114     -23.233     217.893
gender[T.other]    18.2217    142.452      0.128      0.898    -261.199     297.642
sentiment           0.4653      2.598      0.179      0.858      -4.630       5.561
truth               8.7511      9.893      0.885      0.377     -10.654      28.157
urgency             1.2047      3.904      0.309      0.758      -6.453       8.862
parsed_age          0.3771      1.566      0.241      0.810      -2.695       3.449
==============================================================================
Omnibus:                     4408.197   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        117350376.344
Skew:                          35.987   Prob(JB):                         0.00
Kurtosis:                    1349.053   Cond. No.                     4.02e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

time: 45.1 ms (started: 2025-06-30 07:29:05 +12:00)


In [26]:
smf.ols(formula=formula, data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             met_target   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     5.733
Date:                Mon, 30 Jun 2025   Prob (F-statistic):           1.45e-06
Time:                        07:29:05   Log-Likelihood:                -485.41
No. Observations:                1550   AIC:                             986.8
Df Residuals:                    1542   BIC:                             1030.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.2923      0.234     -1.248      0.212      -0.752       0.167
smiling[T.True]     0.0374      0.018      2.076      0.038       0.002       0.073
gender[T.male]      0.0276      0.018      1.566      0.118      -0.007       0.062
gender[T.other]    -0.0525      0.041     -1.285      0.199      -0.133       0.028
sentiment           0.0015      0.001      1.995      0.046     2.5e-05       0.003
truth               0.0035      0.003      1.217      0.224      -0.002       0.009
urgency             0.0004      0.001      0.329      0.742      -0.002       0.003
parsed_age         -0.0018      0.000     -3.909      0.000      -0.003      -0.001
==============================================================================
Omnibus:                      601.932   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1653.654
Skew:                           2.128   Prob(JB):                         0.00
Kurtosis:                       5.738   Cond. No.                     4.02e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

time: 43.8 ms (started: 2025-06-30 07:29:05 +12:00)


In [27]:
model = RandomForestRegressor()
cols = "sentiment + smiling + gender + truth + urgency + ethnicity".split(" + ")
X = pd.get_dummies(df[cols])
y = df["met_target"]
model.fit(X, y)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


time: 523 ms (started: 2025-06-30 07:29:05 +12:00)


In [28]:
list(zip(cols, model.feature_importances_))

[('sentiment', np.float64(0.20825657895338884)),
 ('smiling', np.float64(0.1058103161487281)),
 ('gender', np.float64(0.09467852660130897)),
 ('truth', np.float64(0.21226452837419557)),
 ('urgency', np.float64(0.05075663905860937)),
 ('ethnicity', np.float64(0.050662348021350244))]

time: 11.3 ms (started: 2025-06-30 07:29:05 +12:00)


In [32]:
df.region.value_counts().head(20)

region
Auckland                1331
Canterbury               555
Wellington               489
Waikato                  385
Bay of Plenty            293
Nationwide               210
Otago                    168
Northland                161
Hawke's Bay              134
International            107
Taranaki                 100
Southland                 71
Manawatu-Wanganui         63
Nelson / Tasman           62
Manawatu / Whanganui      51
Manawatu-Whanganui        51
Marlborough               37
Gisborne                  34
Nelson                    28
West Coast                24
Name: count, dtype: int64

time: 9.71 ms (started: 2025-06-30 07:30:09 +12:00)
